# Morel-free RL for Mapless Navigation 
Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
project_root = '/content/drive/My Drive/機器導航與探索/DDPG-Mapless-Navigation-Lab/'
sys.path.append(project_root)

from nav_environment import NavigationEnv
import ddpg
import models
import numpy as np
import os
import eval_ddpg

Parameter setting

In [0]:
batch_size = 64
eval_eps = 50
RL = ddpg.DDPG(
    model = [models.PolicyNet, models.QNet],
    learning_rate = [0.0001, 0.0001],
    reward_decay = 0.99,
    memory_size = 10000,
    batch_size = batch_size)

is_train = True
render = False
load_model = False

Path of the model

In [0]:
gif_path = project_root + "out/"
model_path = project_root + "save/"
if not os.path.exists(model_path):
    os.makedirs(model_path)

if load_model:
    print("Load model ...", model_path)
    RL.save_load_model("load", model_path)

Main function

In [0]:
def main():
    env = NavigationEnv(path = project_root+"Maps/map.png")
    total_step = 0
    max_success_rate = 0
    success_count = 0
    for eps in range(1001):
        state = env.initialize()
        step = 0
        r_eps = []
        loss_a = loss_c = 0.
        acc_reward = 0.
        
        while(True):
            # Choose action and run
            if is_train:
                action = RL.choose_action(state, eval=False)
            else:
                action = RL.choose_action(state, eval=True)
            state_next, reward, done = env.step(action)
            end = 0 if done else 1
            RL.store_transition(state, action, reward, state_next, end)

            # Render Environment
            if render:
                env.render()

            # Record and print information
            r_eps.append(reward)
            acc_reward += reward
            loss_a = loss_c = 0.
            if total_step > batch_size and is_train:
                loss_a, loss_c = RL.learn()

            print('\rEps:{:3d} /{:4d} /{:6d}| action:{:+.2f}| R:{:+.2f}| Loss:[A>{:+.2f} C>{:+.2f}]| Epsilon: {:.3f}| Reps:{:.2f}  '\
                    .format(eps, step, total_step, action[0], reward, loss_a, loss_c, RL.epsilon, acc_reward), end='')
            state = state_next.copy()
            step += 1
            total_step += 1
            if done or step>600:
                print()
                break

        if r_eps[-1] > 2:
            success_count += 1 

        if eps>0 and eps%eval_eps==0:
            # Sucess rate
            success_rate = success_count / eval_eps
            success_count = 0
            
            if success_rate >= max_success_rate:
                max_success_rate = success_rate
                if is_train:
                    print("Save model to " + model_path)
                    RL.save_load_model("save", model_path)

            print("Success Rate:", success_rate, "/", max_success_rate)

            # output GIF
            print("Generate GIF ...")
            eval_ddpg.run(total_eps=4, model_path=model_path, map_path=project_root+"Maps/map.png",\
                gif_path=gif_path, gif_name=str(eps).zfill(4)+"_eps.gif")

In [6]:
main()

Eps:  0 /  65 /    65| action:-0.25| R:-5.29| Loss:[A>-0.02 C>+46.44]| Epsilon: 1.000| Reps:160.52  
Eps:  1 /  46 /   112| action:-0.11| R:-5.45| Loss:[A>-0.01 C>+40.57]| Epsilon: 1.000| Reps:-342.23  
Eps:  2 /   8 /   121| action:+0.32| R:-4.83| Loss:[A>+0.12 C>+34.56]| Epsilon: 0.999| Reps:-48.08  
Eps:  3 /  66 /   188| action:+1.00| R:-5.03| Loss:[A>+0.75 C>+23.01]| Epsilon: 0.999| Reps:45.97  
Eps:  4 /  62 /   251| action:+1.00| R:-4.91| Loss:[A>+1.47 C>+16.29]| Epsilon: 0.998| Reps:281.61  
Eps:  5 /  31 /   283| action:+0.57| R:-5.09| Loss:[A>-1.87 C>+25.20]| Epsilon: 0.998| Reps:100.95  
Eps:  6 /   5 /   289| action:+1.00| R:-5.26| Loss:[A>-0.32 C>+21.80]| Epsilon: 0.998| Reps:-44.61  
Eps:  7 /   7 /   297| action:+0.96| R:-4.95| Loss:[A>+0.73 C>+20.62]| Epsilon: 0.998| Reps:-77.10  
Eps:  8 /  47 /   345| action:+0.90| R:-4.55| Loss:[A>+2.09 C>+17.09]| Epsilon: 0.997| Reps:127.25  
Eps:  9 /   8 /   354| action:+0.67| R:-5.15| Loss:[A>+1.06 C>+12.72]| Epsilon: 0.997| Reps